In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import json
import os
import sklearn as sk

2024-07-24 17:44:23.365729: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 17:44:23.592943: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 17:44:23.594276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 17:44:32.124696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
classes_train = pd.read_csv('../data/train/_classes.csv')
classes_test = pd.read_csv('../data/test/_classes.csv')
classes_valid = pd.read_csv('../data/valid/_classes.csv')

# Concatenate all data
all_data_merged = pd.concat([classes_train, classes_test, classes_valid])

melted = all_data_merged.melt(id_vars=["filename"], var_name="category", value_name="value")
melted = melted[melted["value"] == 1]
all_data_merged_class = melted.drop("value", axis=1).rename(columns={"category": "class"})

all_data_merged_class.head()

subdatasets = [[] for _ in range(5)]

# Create a dictionary to map filenames to their full rows
filename_to_row = {row['filename']: row for _, row in all_data_merged.iterrows()}

# Distribute entries into the subdatasets
for idx, row in all_data_merged_class.iterrows():
    full_row = filename_to_row[row['filename']]
    subdatasets[idx % 5].append(full_row)

# Convert lists to DataFrames
subdatasets = [pd.DataFrame(subdataset) for subdataset in subdatasets]

# Verify the first few rows of each subdataset
for i, subdataset in enumerate(subdatasets):
    subdataset.sample(frac=1).reset_index(drop=True)

#shuffle the subdatasets
for i, subdataset in enumerate(subdatasets):
    subdatasets[i] = subdataset.sample(frac=1).reset_index(drop=True)
    



In [3]:
#write subdatasets to csv files in data/cross_valid_sets
for i, subdataset in enumerate(subdatasets):
    subdataset.to_csv(f'../data/cross_valid_sets/subdataset_{i}.csv', index=False)
    

In [17]:
test_1 = pd.read_csv('../data/cross_valid_sets/subdataset_1.csv')
test_2= pd.read_csv('../data/cross_valid_sets/subdataset_2.csv')
test_3 = pd.read_csv('../data/cross_valid_sets/subdataset_3.csv')

sum_1 = test_1[" SafeDriving"].sum()
sum_2 = test_1[" DangerousDriving"].sum()
sum_3 = test_1[" Yawn"].sum()

sum_4 = classes_train[" SafeDriving"].sum()
sum_5 = classes_train[" DangerousDriving"].sum()
sum_6 = classes_train[" Yawn"].sum()
print(sum_1, sum_2, sum_3)
print(sum_4, sum_5, sum_6)


1231 938 101
4969 3732 439


In [22]:
print(all_data_merged[" SafeDriving"].iloc[5])


0


In [79]:
from sklearn.model_selection import StratifiedKFold
X_aug = np.load('../data/data_aug_2000.npy')
y_aug = pd.read_csv('../data/labels_aug_2000.csv', delimiter=',', index_col=0).to_numpy()
print(y_aug[-1])
labels_aug = []

for i in range(len(y_aug)):
    # print(y_aug[i])
    if 1 in y_aug[i]:
        k = np.where(y_aug[i] == 1)[0][0]
        labels_aug.append(k)
    else:
        labels_aug.append(7)

labels_aug = np.array(labels_aug)
ind = np.where(labels_aug == 7)
print(ind)


kf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)




[0 1 0 0 0 0]
(array([ 3688,  8072,  8769, 10340]),)


In [85]:
cross_ind = []

for train_index, val_index in kf.split(X = X_aug, y = labels_aug):
    # train_data, val_data = X_aug[train_index], X_aug[val_index]
    # train_labels, val_labels = y_aug[train_index], y_aug[val_index]
    cross_ind.append(val_index)

cross_ind = np.array(cross_ind)
print(cross_ind)

[array([    3,     4,     8, ..., 11986, 11987, 11990])
 array([    7,    11,    20, ..., 11968, 11972, 12001])
 array([   16,    19,    27, ..., 11958, 11978, 11991])
 array([    2,     5,    10, ..., 11997, 12002, 12003])
 array([    6,    17,    23, ..., 11995, 11996, 12000])
 array([    0,     1,    32, ..., 11979, 11998, 11999])]


/home/maxm/conda/envs/default/lib/python3.10/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=6.
  warnings.warn(
/tmp/ipykernel_8905/3001742484.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cross_ind = np.array(cross_ind)


In [92]:
print(len(X_aug[cross_ind[0]]))
labels_1_6 = y_aug[cross_ind[0]]
print(labels_1_6.sum(axis = 0))

2001
[333 334 333 333 333 334]


In [93]:
np.save('../data/cross_val_6fold_2000.npy', cross_ind)